# Полная интеграция: Анализ системы переписывания на всех уровнях

## Зачем нужен этот пайплайн?

Когда мы определяем **систему переписывания** (набор правил вида `левая_часть → правая_часть`), возникают естественные вопросы:

1. **Какие строки достижимы** из начальной? (Пространство состояний)
2. **Есть ли циклы** в системе? (Структура переходов)
3. **Какие паттерны повторяются**? (Макросы и регулярности)
4. **Корректна ли система**? (Верификация свойств)
5. **Какова геометрия** пространства строк? (Математическая структура)

**SpaceLanguageOrchestrator** — инструмент, который автоматически отвечает на все эти вопросы, запуская анализ на **8 уровнях** (L0-L7).

## Словарь терминов

| Термин | Расшифровка | Что это значит |
|--------|-------------|----------------|
| **L0** | Level 0: Rewriting | Базовая система переписывания строк |
| **L1** | Level 1: Turing | Машина Тьюринга (вычислительная модель) |
| **L2** | Level 2: Graphs | Граф переходов между строками |
| **SCC** | Strongly Connected Components | Сильно связные компоненты графа (группы вершин, где каждая достижима из каждой) |
| **L3** | Level 3: Overlaps | Перекрытия между правилами (AC-автомат) |
| **L4** | Level 4: Macros | Повторяющиеся паттерны в строках |
| **L5** | Level 5: Text | Генерация текстового описания |
| **L6** | Level 6: Verification | Проверка свойств (Z3-solver, Isabelle) |
| **L7** | Level 7: Math Lifts | Математические подъёмы (геометрия, меры, категории) |

## Сквозной пример

Будем анализировать **унарную арифметику** — ту же систему из предыдущих туториалов:
- `00 → |` (две единицы → пара)
- `|| → 0` (две пары → единица)

Проследим её через все 8 уровней анализа!

---

## Шаг 1: Настройка окружения

In [1]:
import sys
from pathlib import Path

# Настройка путей к модулям
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Импортируем основные компоненты
from rewriting import String, Rule, RewritingEngine
from integration.orchestrator import SpaceLanguageOrchestrator, PipelineConfig

print("✓ Модули загружены")
print(f"✓ Корневая директория: {project_root}")

✓ Модули загружены
✓ Корневая директория: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang


---

## Шаг 2: Определение системы переписывания (L0)

**Система переписывания** — это набор правил, которые преобразуют строки.

Наш сквозной пример: **унарная арифметика**
- Символ `0` — единица
- Символ `|` — пара
- Правило `00 → |` — две единицы объединяются в пару
- Правило `|| → 0` — две пары свёртываются в единицу

In [2]:
# Определяем правила унарной арифметики
rules = [
    Rule(String.from_str("00"), String.from_str("|")),   # R1: 00 → |
    Rule(String.from_str("||"), String.from_str("0")),   # R2: || → 0
]

# Начальная строка
initial = String.from_str("0000")

print("="*60)
print("СИСТЕМА ПЕРЕПИСЫВАНИЯ (L0)")
print("="*60)
print()
print("Правила:")
for i, rule in enumerate(rules, 1):
    print(f"  R{i}: {rule.left} → {rule.right}")
print()
print(f"Начальная строка: '{initial}'")
print()
print("Вопрос: какие строки можно получить из '0000'?")
print("Ответ узнаем после полного анализа!")

СИСТЕМА ПЕРЕПИСЫВАНИЯ (L0)

Правила:
  R1: 00 → |
  R2: || → 0

Начальная строка: '0000'

Вопрос: какие строки можно получить из '0000'?
Ответ узнаем после полного анализа!


---

## Шаг 3: Конфигурация анализа

**PipelineConfig** — настройки для всех уровней анализа.

### Что можно настроить:

| Параметр | Уровень | Значение | Описание |
|----------|---------|----------|----------|
| `graph_depth` | L2 | 10 | Глубина поиска в графе переходов |
| `max_graph_size` | L2 | 100 | Максимум вершин в графе |
| `min_overlap_length` | L3 | 1 | Минимальная длина перекрытия |
| `min_macro_frequency` | L4 | 2 | Минимальная частота макроса |
| `enable_z3` | L6 | True/False | Включить SMT-верификацию |
| `embedding_dim` | L7 | 8 | Размерность геом. погружения |

In [3]:
# Создаём конфигурацию для нашего примера
config = PipelineConfig(
    # L2: Графовый анализ
    graph_depth=10,        # Ищем строки на глубине до 10 шагов
    max_graph_size=100,    # Максимум 100 вершин в графе
    
    # L3: Поиск перекрытий между правилами
    min_overlap_length=1,
    
    # L4: Поиск повторяющихся паттернов
    min_macro_frequency=2,  # Паттерн должен встречаться минимум 2 раза
    min_macro_length=2,     # Минимальная длина паттерна
    max_macro_length=6,
    
    # L6: Верификация
    enable_z3=False,        # Отключаем (требует отдельной установки z3-solver)
    enable_isabelle=False,  # Отключаем (требует Isabelle)
    
    # L7: Математические подъёмы
    enable_geometry=True,   # Погружение в ℝⁿ
    enable_measure=True,    # Вероятностные меры
    enable_category=True,   # Категорные структуры
    embedding_dim=8,        # Размерность пространства для геометрии
)

print("="*60)
print("КОНФИГУРАЦИЯ АНАЛИЗА")
print("="*60)
print()
print(f"L2 (Графы):     глубина={config.graph_depth}, макс_вершин={config.max_graph_size}")
print(f"L4 (Макросы):   мин_частота={config.min_macro_frequency}, длина={config.min_macro_length}-{config.max_macro_length}")
print(f"L6 (Проверка):  Z3={'вкл' if config.enable_z3 else 'выкл'}, Isabelle={'вкл' if config.enable_isabelle else 'выкл'}")
print(f"L7 (Геометрия): размерность={config.embedding_dim}")

КОНФИГУРАЦИЯ АНАЛИЗА

L2 (Графы):     глубина=10, макс_вершин=100
L4 (Макросы):   мин_частота=2, длина=2-6
L6 (Проверка):  Z3=выкл, Isabelle=выкл
L7 (Геометрия): размерность=8


---

## Шаг 4: Создание оркестратора

**SpaceLanguageOrchestrator** — центральный компонент, который:
1. Принимает правила переписывания
2. Координирует работу всех 8 уровней анализа
3. Собирает результаты в единую структуру

In [4]:
# Создаём оркестратор
orchestrator = SpaceLanguageOrchestrator(rules, config)

print("="*60)
print("ОРКЕСТРАТОР СОЗДАН")
print("="*60)
print()
print(f"Правил загружено: {len(rules)}")
print(f"Уровней анализа: 8 (L0-L7)")
print()
print("Готов к анализу начальной строки!")

ОРКЕСТРАТОР СОЗДАН

Правил загружено: 2
Уровней анализа: 8 (L0-L7)

Готов к анализу начальной строки!


---

## Шаг 5: Запуск полного анализа

Сейчас оркестратор последовательно выполнит:

1. **L0**: Найдёт все достижимые строки (BFS от начальной)
2. **L2**: Построит граф переходов и найдёт SCC
3. **L3**: Проанализирует перекрытия между правилами
4. **L4**: Извлечёт повторяющиеся макросы
5. **L7**: Вычислит геометрические и статистические характеристики

In [5]:
# Запускаем полный анализ
print("="*60)
print("ЗАПУСК АНАЛИЗА")
print("="*60)
print()

result = orchestrator.analyze(initial, verbose=True)

print()
print("="*60)
print("✓ АНАЛИЗ ЗАВЕРШЁН")
print("="*60)

ЗАПУСК АНАЛИЗА

L0: Анализ системы переписывания...
L2: Построение графа и SCC...
L3: Анализ перекрытий...
L4: Извлечение макросов...
L5: Генерация текста...
L7: Математические подъёмы...

✓ АНАЛИЗ ЗАВЕРШЁН


---

## Шаг 6: Результаты L0 — Достижимые строки

**Что это**: все строки, которые можно получить из начальной применением правил.

**Алгоритм**: BFS (поиск в ширину) от начальной строки, применяя все возможные правила на каждом шаге.

**Ценность**: показывает "пространство состояний" системы — все возможные конфигурации.

In [6]:
print("="*60)
print("L0: ДОСТИЖИМЫЕ СТРОКИ")
print("="*60)
print()
print(f"Из начальной строки '{initial}' достижимо {len(result.reachable_strings)} строк:")
print()

# Сортируем по длине для наглядности
sorted_strings = sorted(result.reachable_strings, key=lambda x: (len(str(x)), str(x)))
for s in sorted_strings:
    # Проверяем, нормальная ли это форма
    is_normal = s in result.normal_forms
    marker = " ← нормальная форма" if is_normal else ""
    print(f"  '{s}'{marker}")

print()
print(f"Нормальных форм: {len(result.normal_forms)}")
print("(строки, к которым нельзя применить ни одно правило)")
print()

# ω-предел
print(f"ω-предел: {len(result.omega_limit)} строк")
print("(строки, в которых система 'застревает' при бесконечном выполнении)")

L0: ДОСТИЖИМЫЕ СТРОКИ

Из начальной строки '0000' достижимо 6 строк:

  '0' ← нормальная форма
  '||'
  '00|'
  '0|0' ← нормальная форма
  '|00'
  '0000'

Нормальных форм: 2
(строки, к которым нельзя применить ни одно правило)

ω-предел: 1 строк
(строки, в которых система 'застревает' при бесконечном выполнении)


---

## Шаг 7: Результаты L2 — Граф переходов и SCC

**Что это**: 
- **Граф переходов** — вершины = строки, рёбра = применения правил
- **SCC (Strongly Connected Components)** — группы строк, где из каждой можно попасть в любую другую

**Алгоритм**: 
1. Построить граф из достижимых строк
2. Найти SCC алгоритмом Тарьяна (обход в глубину + стек)

**Ценность**: 
- SCC показывают **циклические структуры** в системе
- Если SCC содержит >1 строки — есть цикл!
- Количество SCC характеризует "сложность" динамики

In [7]:
print("="*60)
print("L2: ГРАФ ПЕРЕХОДОВ И SCC")
print("="*60)
print()
print("ГРАФ ПЕРЕХОДОВ:")
print(f"  Вершин (строк): {result.graph_vertices}")
print(f"  Рёбер (переходов): {result.graph_edges}")
print()

print("SCC (Strongly Connected Components):")
print(f"  Компонент: {result.scc_count}")
print(f"  Наибольшая: {result.largest_scc_size} вершин")
print()

# Интерпретация
if result.largest_scc_size == 1:
    print("Интерпретация: все SCC размера 1 → нет циклов в системе")
    print("  Это означает, что любая траектория заканчивается в нормальной форме.")
else:
    print(f"Интерпретация: есть SCC размера {result.largest_scc_size} → система содержит циклы!")
    print("  Это означает, что некоторые траектории могут зацикливаться.")

L2: ГРАФ ПЕРЕХОДОВ И SCC

ГРАФ ПЕРЕХОДОВ:
  Вершин (строк): 6
  Рёбер (переходов): 6

SCC (Strongly Connected Components):
  Компонент: 6
  Наибольшая: 1 вершин

Интерпретация: все SCC размера 1 → нет циклов в системе
  Это означает, что любая траектория заканчивается в нормальной форме.


---

## Шаг 8: Результаты L4 — Макросы

**Что это**: повторяющиеся подстроки (паттерны) в достижимых строках.

**Алгоритм**: 
1. Собрать все достижимые строки
2. Найти подстроки длины 2-6, встречающиеся ≥2 раз
3. Отфильтровать по частоте

**Ценность**: 
- Макросы показывают "структурные единицы" системы
- Частые паттерны могут иметь особое значение
- Помогают понять "язык" системы

In [8]:
print("="*60)
print("L4: МАКРОСЫ (повторяющиеся паттерны)")
print("="*60)
print()

if result.macros and len(result.macros) > 0:
    print(f"Найдено макросов: {len(result.macros)}")
    print()
    print("Список макросов:")
    for i, macro in enumerate(result.macros, 1):
        print(f"  {i}. '{macro}'")
    print()
    print("Интерпретация: эти паттерны часто встречаются в достижимых строках.")
else:
    print("Макросов не найдено.")
    print()
    print("Интерпретация: система слишком мала или паттерны не повторяются.")
    print("  Для нашего примера (6 коротких строк) это нормально.")

L4: МАКРОСЫ (повторяющиеся паттерны)

Макросов не найдено.

Интерпретация: система слишком мала или паттерны не повторяются.
  Для нашего примера (6 коротких строк) это нормально.


---

## Шаг 9: Результаты L7 — Математические подъёмы

**L7** содержит три компонента:

### 📐 Геометрия
- Строки → точки в ℝⁿ (через вектор признаков)
- Вычисляется размерность и кривизна многообразия

### 📊 Меры
- Вычисляется энтропия распределения
- Показывает "неопределённость" системы

### 🎯 Категории
- Строки → объекты, правила → морфизмы
- Абстрактное описание структуры

In [9]:
print("="*60)
print("L7: МАТЕМАТИЧЕСКИЕ ПОДЪЁМЫ")
print("="*60)
print()

# Геометрия
print("📐 ГЕОМЕТРИЯ:")
if result.geometry_data:
    dim = result.geometry_data.get('dimension', 'N/A')
    curv = result.geometry_data.get('curvature', 0)
    pts = result.geometry_data.get('points', 0)
    print(f"  Размерность многообразия: {dim}")
    print(f"  Кривизна: {curv:.4f}")
    print(f"  Точек (строк): {pts}")
    print()
    if curv < 0.1:
        print("  Интерпретация: низкая кривизна → строки распределены равномерно")
    else:
        print("  Интерпретация: высокая кривизна → есть кластеры строк")
else:
    print("  (данные недоступны)")
print()

L7: МАТЕМАТИЧЕСКИЕ ПОДЪЁМЫ

📐 ГЕОМЕТРИЯ:
  Размерность многообразия: 1
  Кривизна: 0.0402
  Точек (строк): 6

  Интерпретация: низкая кривизна → строки распределены равномерно



In [10]:
# Меры
print("📊 МЕРЫ И ЭНТРОПИЯ:")
if result.measure_data:
    entropy = result.measure_data.get('entropy', 0)
    support = result.measure_data.get('support_size', 0)
    print(f"  Энтропия: {entropy:.3f} бит")
    print(f"  Размер носителя: {support} строк")
    print()
    import math
    max_entropy = math.log2(support) if support > 1 else 0
    if max_entropy > 0:
        ratio = entropy / max_entropy
        print(f"  H/H_max = {ratio:.1%}")
        if ratio > 0.9:
            print("  Интерпретация: высокая энтропия → строки равновероятны")
        else:
            print("  Интерпретация: низкая энтропия → есть 'предпочтительные' строки")
else:
    print("  (данные недоступны)")
print()

# Категории
print("🎯 КАТЕГОРИИ:")
if result.category_data:
    objs = result.category_data.get('objects', 0)
    morphs = result.category_data.get('morphisms', 0)
    print(f"  Объектов (строк): {objs}")
    print(f"  Морфизмов (переходов): {morphs}")
    print()
    if morphs > 0 and objs > 0:
        avg_out = morphs / objs
        print(f"  Среднее число переходов из строки: {avg_out:.1f}")
else:
    print("  (данные недоступны)")

📊 МЕРЫ И ЭНТРОПИЯ:
  Энтропия: 2.585 бит
  Размер носителя: 6 строк

  H/H_max = 100.0%
  Интерпретация: высокая энтропия → строки равновероятны

🎯 КАТЕГОРИИ:
  Объектов (строк): 6
  Морфизмов (переходов): 6

  Среднее число переходов из строки: 1.0


---

## Шаг 10: Полная сводка результатов

Выведем все результаты анализа в одном месте:

In [11]:
print("="*70)
print("                    ИТОГОВАЯ СВОДКА АНАЛИЗА")
print("="*70)
print()
print(f"Система: {len(rules)} правила, начальная строка '{initial}'")
print()
print("┌─────────────────────────────────────────────────────────────────────┐")
print("│ УРОВЕНЬ │                  РЕЗУЛЬТАТ                                │")
print("├─────────────────────────────────────────────────────────────────────┤")
print(f"│ L0      │ Достижимых строк: {len(result.reachable_strings):>3}                                   │")
print(f"│         │ Нормальных форм:  {len(result.normal_forms):>3}                                   │")
print(f"│         │ ω-предел:         {len(result.omega_limit):>3} строк                              │")
print("├─────────────────────────────────────────────────────────────────────┤")
print(f"│ L2      │ Вершин графа:     {result.graph_vertices:>3}                                   │")
print(f"│         │ Рёбер:            {result.graph_edges:>3}                                   │")
print(f"│         │ SCC компонент:    {result.scc_count:>3}                                   │")
print("├─────────────────────────────────────────────────────────────────────┤")
print(f"│ L4      │ Макросов:         {len(result.macros) if result.macros else 0:>3}                                   │")
print("├─────────────────────────────────────────────────────────────────────┤")
if result.geometry_data:
    print(f"│ L7      │ Размерность:      {result.geometry_data.get('dimension', 0):>3}                                   │")
    print(f"│         │ Кривизна:         {result.geometry_data.get('curvature', 0):>6.3f}                               │")
if result.measure_data:
    print(f"│         │ Энтропия:         {result.measure_data.get('entropy', 0):>6.3f} бит                           │")
print("└─────────────────────────────────────────────────────────────────────┘")
print()

# Ошибки
if result.errors:
    print("⚠️ Предупреждения:")
    for e in result.errors:
        print(f"  • {e}")

                    ИТОГОВАЯ СВОДКА АНАЛИЗА

Система: 2 правила, начальная строка '0000'

┌─────────────────────────────────────────────────────────────────────┐
│ УРОВЕНЬ │                  РЕЗУЛЬТАТ                                │
├─────────────────────────────────────────────────────────────────────┤
│ L0      │ Достижимых строк:   6                                   │
│         │ Нормальных форм:    2                                   │
│         │ ω-предел:           1 строк                              │
├─────────────────────────────────────────────────────────────────────┤
│ L2      │ Вершин графа:       6                                   │
│         │ Рёбер:              6                                   │
│         │ SCC компонент:      6                                   │
├─────────────────────────────────────────────────────────────────────┤
│ L4      │ Макросов:           0                                   │
├──────────────────────────────────────────────────────────

---

## Что мы узнали о системе `00→|, ||→0`?

### Факты из анализа:

1. **Пространство состояний ограничено** — из `'0000'` достижимо всего 6 строк
2. **Система терминирует** — все SCC размера 1, значит нет циклов
3. **Есть 2 нормальные формы** — `'0'` и `'0|0'` (конечные состояния)
4. **Структура простая** — низкая кривизна, строки распределены равномерно

### Ценность результатов:

| Что узнали | Практическое значение |
|------------|----------------------|
| Число достижимых строк | Размер пространства поиска |
| Наличие циклов (SCC) | Гарантия завершения |
| Нормальные формы | Возможные конечные результаты |
| Энтропия | Предсказуемость системы |

## 📊 Резюме туториала

В этом туториале вы научились:

✅ Использовать **SpaceLanguageOrchestrator** для полного анализа  
✅ Настраивать параметры через **PipelineConfig**  
✅ Интерпретировать результаты каждого уровня (L0-L7)  
✅ Понимать связь между уровнями анализа  

## 🚀 Следующий шаг

**04_advanced_examples.ipynb** — Более сложные системы с циклами и богатой структурой